In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv(f"../data/bbc_text_cls.csv")
print(df['labels'].value_counts())
df = df[df['labels'] == 'business']
print(df['labels'].value_counts())

labels
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64
labels
business    510
Name: count, dtype: int64


In [3]:
df.drop(columns='labels', inplace=True)
df.head()

,text
0,Ad sales boost Time Warner profit\n\nQuarterly...
1,Dollar gains on Greenspan speech\n\nThe dollar...
2,Yukos unit buyer faces loan claim\n\nThe owner...
3,High fuel prices hit BA's profits\n\nBritish A...
4,Pernod takeover talk lifts Domecq\n\nShares in...


In [4]:
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace(r'\n', ' ', regex=True)
df['text'] = df['text'].str.replace(r'[^\w\s]', '', regex=True)
df['text'] = df['text'].str.replace(r'[\d]', '', regex=True)
df['text_list'] = df['text'].apply(lambda x: x.split())
df.head()

,text,text_list
0,ad sales boost time warner profit quarterly p...,"[ad, sales, boost, time, warner, profit, quart..."
1,dollar gains on greenspan speech the dollar h...,"[dollar, gains, on, greenspan, speech, the, do..."
2,yukos unit buyer faces loan claim the owners ...,"[yukos, unit, buyer, faces, loan, claim, the, ..."
3,high fuel prices hit bas profits british airw...,"[high, fuel, prices, hit, bas, profits, britis..."
4,pernod takeover talk lifts domecq shares in u...,"[pernod, takeover, talk, lifts, domecq, shares..."


In [5]:
#unique word list
word_set = set()

for text in df['text_list']:
    word_set.update(text)
idx2word = list(word_set)
idx2word.insert(0, '<UNKNOWN>')

word2idx = {word:i for i, word in enumerate(idx2word)}
print(len(word2idx))
print(list(word2idx.items())[0:5])

11963
[('<UNKNOWN>', 0), ('newsome', 1), ('expecting', 2), ('assumed', 3), ('spent', 4)]


In [6]:
def word_numerizer(arr):
    return [word2idx.get(word, 0) for word in arr]

In [7]:
df['text_list_int'] = df['text_list'].apply(lambda x: word_numerizer(x))
df['text_list_int']

0      [9984, 367, 11619, 10651, 1120, 9938, 9471, 36...
1      [3267, 6976, 8010, 5924, 10002, 2689, 3267, 15...
2      [7266, 3627, 9865, 5511, 2616, 6556, 2689, 6, ...
3      [8133, 5914, 2492, 3963, 7298, 3608, 940, 1315...
4      [11171, 6008, 7359, 5933, 6087, 11538, 3913, 2...
                             ...                        
505    [895, 4955, 10383, 4498, 3249, 3629, 2689, 895...
506    [2142, 6677, 6230, 8535, 10800, 883, 2689, 214...
507    [7702, 1054, 1185, 8010, 895, 2689, 451, 1623,...
508    [7531, 1161, 9849, 7361, 8010, 1927, 3946, 731...
509    [11636, 2147, 6987, 358, 8826, 7912, 1724, 776...
Name: text_list_int, Length: 510, dtype: object

In [12]:
# Prepare A1, A2 and pi

pi = {}
A1 = {}
A2 = {}

for sentence in df['text_list_int']: 
    #pi
    if sentence[0] not in pi:
        pi[sentence[0]] = 1
    else:
        pi[sentence[0]] +=1
    #A1
    if len(sentence) >=2:
        first_word = sentence[0]
        second_word = sentence[1]
        if first_word not in A1:
            A1[first_word] = {}
            A1[first_word][second_word] = 1
        else:
            if second_word not in A1[first_word]:
                A1[first_word][second_word] = 1
            else:
                A1[first_word][second_word] +=1 
    #A2
    for word_index in range(len(sentence)-2):
        #key
        current_word = sentence[word_index]
        second_word  = sentence[word_index+1]
        third_word   = sentence[word_index+2]

        keytuple = (current_word, third_word)

        if keytuple not in A2:
            A2[keytuple] = {}
            A2[keytuple][second_word] = 1                    
        else:
            if second_word not in A2[keytuple]:
                A2[keytuple][second_word]= 1
            else:
                A2[keytuple][second_word]+= 1